## Exercício Desafio

- Digamos que seu chefe pediu para você um relatório da análise dos salários da unidade de San Francisco da empresa. O objetivo dele é entender:

1. Qual foi a evolução do salário médio ao longo dos anos (TotalPay e TotalPayBenefits)
2. Quantos funcionários tivemos ao longo dos anos
3. Qual foi a evolução do total gasto com salário ao longo dos anos (TotalPayBenefits)

- Base de Dados a ser usada: salarios.sqlite

In [ ]:
qtd_funcionarios = len(df_salarios['EmployeeName'].unique())
qtd_funcionarios

### Importação da Base de Dados

In [14]:
import pyodbc
import pandas as pd

dados_conexao = ("Driver={SQLite3 ODBC Driver};"
                 "Service=localhost;"
                 "Database=salarios.sqlite")


conexao = pyodbc.connect(dados_conexao)

cursor = conexao.cursor()

In [15]:
cursor.execute("SELECT * FROM Salaries")

valores = cursor.fetchall()
cabecalho = [coluna[0] for coluna in cursor.description]
# print(cabecalho)

['Id', 'EmployeeName', 'JobTitle', 'BasePay', 'OvertimePay', 'OtherPay', 'Benefits', 'TotalPay', 'TotalPayBenefits', 'Year', 'Notes', 'Agency', 'Status']


In [16]:
cursor.close()
conexao.close()

In [17]:
df_salarios = pd.DataFrame.from_records(valores, columns=cabecalho)
# df_salarios = df_salarios[df_salarios['Agency'] == 'San Francisco']

df_salarios.head(2)

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.00,400184.25,NaN,567595.43,567595.43,2011,,San Francisco,
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,NaN,538909.28,538909.28,2011,,San Francisco,


### Análise de Dados

In [19]:
# garantindo que estamos só com San Francisco
df_salarios_filtr = df_salarios[df_salarios['Agency'] == 'San Francisco']
df_salarios_filtr.head(2)

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.00,400184.25,NaN,567595.43,567595.43,2011,,San Francisco,
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,NaN,538909.28,538909.28,2011,,San Francisco,


##### 1. Qual foi a evolução do salário médio ao longo dos anos

In [22]:
df_salarios_por_ano = df_salarios.groupby('Year')[['TotalPay', 'TotalPayBenefits']].mean()

# df_salarios_por_ano.plot(kind='bar')
df_salarios_por_ano

,TotalPay,TotalPayBenefits
Year,,
2011,71744.103871,71744.103871
2012,74113.262265,100553.229232
2013,77611.443142,101440.519714
2014,75463.918140,100250.918884


##### 2. Quantos funcionários tivemos ao longo dos anos

In [30]:
df_tot_func_por_ano = df_salarios.groupby('Year')[['Id']].count()
df_tot_func_por_ano = df_tot_func_por_ano.rename(columns={'Id': 'Qtd Funcionarios'})
df_tot_func_por_ano

,Qtd Funcionarios
Year,
2011,36159
2012,36766
2013,37606
2014,38123


##### 3. Qual foi a evolução do total gasto com salário ao longo dos anos

In [37]:
def formatar(valor):
    return f'R$ {valor:,.2f}'.replace(',', '_').replace('.', ',').replace('_', '.')


df_total_sal = df_salarios.groupby('Year')[['TotalPay', 'TotalPayBenefits']].sum(numeric_only=True)
df_total_sal['TotalPay'] = df_total_sal['TotalPay'].map(formatar)
df_total_sal['TotalPayBenefits'] = df_total_sal['TotalPayBenefits'].apply(formatar)

df_total_sal

,TotalPay,TotalPayBenefits
Year,,
2011,"R$ 2.594.195.051,88","R$ 2.594.195.051,88"
2012,"R$ 2.724.848.200,44","R$ 3.696.940.025,96"
2013,"R$ 2.918.655.930,80","R$ 3.814.772.184,37"
2014,"R$ 2.876.910.951,26","R$ 3.821.865.780,60"
